In [1]:
import numpy as np

# Load the data from the provided .npz file
data = np.load('output/2D/2023cvpr_keypoints.npz')
print(data.files)  # Inspect the keys in the loaded data


['reconstruction']


In [2]:
# Inspect the shape and contents of the 'reconstruction' dataset
reconstruction_data = data['reconstruction']
reconstruction_data.shape


(177, 17, 3)

In [3]:
# Inspect the shape and some initial values of the 'reconstruction' dataset
reconstruction_data = data['reconstruction']
shape = reconstruction_data.shape
sample_values = reconstruction_data[:2]  # Display the first two sets of values for inspection

shape, sample_values

((177, 17, 3),
 array([[[ 0.        ,  0.        ,  0.        ],
         [-0.08508728, -0.01492222,  0.05849564],
         [ 0.01885218,  0.41483778,  0.14102304],
         [ 0.07451244,  0.80502903,  0.30140513],
         [ 0.08677348,  0.0057563 , -0.04997707],
         [ 0.09547751,  0.42339265,  0.02187591],
         [ 0.11455063,  0.83232725,  0.14395282],
         [ 0.02389754, -0.20774162, -0.04282127],
         [-0.01427492, -0.4724248 , -0.06280509],
         [-0.054504  , -0.54667723, -0.1510269 ],
         [-0.02021894, -0.6391061 , -0.08404957],
         [ 0.10939094, -0.4334141 , -0.13120651],
         [ 0.15689057, -0.15260759, -0.14643593],
         [ 0.0841378 ,  0.09489036, -0.17914793],
         [-0.07922047, -0.43125433,  0.02514104],
         [-0.10756294, -0.16538352,  0.11434394],
         [-0.13914795,  0.06644435,  0.00690679]],
 
        [[ 0.        ,  0.        ,  0.        ],
         [-0.08508177, -0.01502779,  0.05847872],
         [ 0.01884692,  0.414852

In [4]:
import plotly.graph_objs as go
from ipywidgets import interact

# 骨骼连接数据
connections = [
    [0, 1], [1, 2], [2, 3], [0, 4], [4, 5],
    [5, 6], [0, 7], [7, 8], [8, 9], [9, 10],
    [8, 11], [11, 12], [12, 13], [8, 14], [14, 15], [15, 16]
]
# 点的标签


In [5]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets

In [6]:
def calculate_angle_between_vectors(v1, v2):
    """Calculate the angle in degrees between two vectors."""
    unit_vector_1 = v1 / np.linalg.norm(v1)
    unit_vector_2 = v2 / np.linalg.norm(v2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    return np.degrees(angle)

# 假设有方法获取关节位置
def get_joint_position(frame_data, index):
    return frame_data[index]

# 添加角度显示到图形中
def add_angle_display(fig, pos1, pos2, pos3, label):
    vector1 = pos2 - pos1
    vector2 = pos2 - pos3
    angle = calculate_angle_between_vectors(vector1, vector2)
    angle_text = f"{label} Angle: {angle:.2f}°"
    fig.add_trace(go.Scatter3d(
        x=[pos2[0]],  
        y=[pos2[2]],  # 适应您的轴设置
        z=[-pos2[1]],
        text=[angle_text],
        mode='text',
        textposition="top center"
    ))

# 添加角度显示到图形中
def add_angle_displayV2(fig, pos1, pos2, vector2, label):
    vector1 = pos2 - pos1
    vector2 = vector2
    angle = calculate_angle_between_vectors(vector1, vector2)
    angle_text = f"{label} Angle: {angle:.2f}°"
    fig.add_trace(go.Scatter3d(
        x=[pos2[0]],  
        y=[pos2[2]],  # 适应您的轴设置
        z=[-pos2[1]],
        text=[angle_text],
        mode='text',
        textposition="top center"
    ))

In [8]:
# 初始化FigureWidget
fig = go.FigureWidget()

# 设置布局
fig.update_layout(scene=dict(
                    xaxis=dict(title='X'),
                    yaxis=dict(title='Y'),  # 确保y轴垂直方向
                    zaxis=dict(title='Z')
                ))


labels = ['Hips', 'LeftThight', 'RightShin', 'RightFoot', 'RightThigh', 'LeftShin', 'LeftFoot', 'Spine', 'Neck', 'UpperNeck', 'TopHead', 'LeftShoulder', 'LeftArm', 'LeftHand', 'RightShoulder', 'RightArm', 'RightHand']

# 为FigureWidget添加初始帧的数据
def init_figure():
    frame_data = reconstruction_data[0]
    for connection in connections:
        x = [frame_data[connection[0], 0], frame_data[connection[1], 0]]
        y = [frame_data[connection[0], 2], frame_data[connection[1], 2]]  # 反转原z轴的值
        z = [-frame_data[connection[0], 1], -frame_data[connection[1], 1]]  # 使用原y轴的值
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='blue'), showlegend=False))
    # fig.add_trace(go.Scatter3d(x=frame_data[:, 0], y=frame_data[:, 2], z=-frame_data[:, 1], mode='markers', marker=dict(size=5, color='red')))

    # 示例：计算并显示大腿和小腿的角度
    right_thigh_pos = get_joint_position(frame_data, labels.index('RightThigh'))
    right_shin_pos = get_joint_position(frame_data, labels.index('RightShin'))
    right_foot_pos = get_joint_position(frame_data, labels.index('RightFoot'))
    
    add_angle_display(fig, right_thigh_pos, right_shin_pos, right_foot_pos, "Right Leg")


    # 示例：计算并显示大腿和小腿的角度
    Hips_pos = get_joint_position(frame_data, labels.index('Hips'))
    Head_pos = get_joint_position(frame_data, labels.index('TopHead'))
    vertical_vector =  np.array([0, 0, 1])  # 取默认的垂直向量 
    
    add_angle_displayV2(fig, Head_pos, Hips_pos, vertical_vector, "Body")

    # 对于每个标签添加一个点到图例
    for i, label in enumerate(labels):
        fig.add_trace(go.Scatter3d(
            x=[frame_data[i, 0]],
            y=[frame_data[i, 2]],
            z=[-frame_data[i, 1]],
            mode='markers',
            marker=dict(size=5, color='red'),  # 指定颜色为红色            
            name=label,
        ))
init_figure()

# 创建滑动条
slider = widgets.IntSlider(min=0, max=reconstruction_data.shape[0] - 1, step=1, value=0)

# 滑动条的回调函数
def update_frame(change):
    frame_data = reconstruction_data[change.new]
    with fig.batch_update():
        fig.data = []
        for connection in connections:
            x = [frame_data[connection[0], 0], frame_data[connection[1], 0]]
            y = [frame_data[connection[0], 2], frame_data[connection[1], 2]]  # 反转原z轴的值
            z = [-frame_data[connection[0], 1], -frame_data[connection[1], 1]]  # 使用原y轴的值
            fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='blue'), showlegend=False))
        # fig.add_trace(go.Scatter3d(x=frame_data[:, 0], y=frame_data[:, 2], z=-frame_data[:, 1], mode='markers', marker=dict(size=5, color='red')))

    # 示例：计算并显示大腿和小腿的角度
    right_thigh_pos = get_joint_position(frame_data, labels.index('RightThigh'))
    right_shin_pos = get_joint_position(frame_data, labels.index('RightShin'))
    right_foot_pos = get_joint_position(frame_data, labels.index('RightFoot'))
    
    add_angle_display(fig, right_thigh_pos, right_shin_pos, right_foot_pos, "Right Leg")

    # 示例：计算并显示大腿和小腿的角度
    Hips_pos = get_joint_position(frame_data, labels.index('Hips'))
    Head_pos = get_joint_position(frame_data, labels.index('Neck'))
    vertical_vector =  np.array([0, 1, 0])  # 取默认的垂直向量 
    
    add_angle_displayV2(fig, Head_pos, Hips_pos, vertical_vector, "Body")

    # 对于每个标签添加一个点到图例
    for i, label in enumerate(labels):
        fig.add_trace(go.Scatter3d(
            x=[frame_data[i, 0]],
            y=[frame_data[i, 2]],
            z=[-frame_data[i, 1]],
            mode='markers',
            marker=dict(size=5, color='red'),  # 指定颜色为红色            
            name=label,
        ))
slider.observe(update_frame, names='value')

display(slider, fig)

IntSlider(value=0, max=176)

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter3d',
              'uid': '7880fe3c-8d51-499e-84cc-ca9430220ebf',
              'x': [0.0, -0.08508728444576263],
              'y': [0.0, 0.05849564075469971],
              'z': [-0.0, 0.014922218397259712]},
             {'line': {'color': 'blue'},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter3d',
              'uid': '8899a863-9ec2-4433-b0ba-edce1e456559',
              'x': [-0.08508728444576263, 0.01885218359529972],
              'y': [0.05849564075469971, 0.14102303981781006],
              'z': [0.014922218397259712, -0.4148377776145935]},
             {'line': {'color': 'blue'},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter3d',
              'uid': 'ba58553b-1604-47a1-b48c-d990ccff2ab1',
              'x': [0.01885218359529972